In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import re

import h5py
from tqdm import tqdm

In [6]:
# this will load the data from the hdf5 file and format it as a pandas dataframe
# the hdf5 files need to be in the current directory (same as the notebook)
# ONLY RUN THIS CELL ONCE TO GET THE CSV FILES

from utils import load_and_format_data_as_df

for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        if '.hdf5' in filename:
            print(f'loading {path}')
            df = load_and_format_data_as_df(path)
            df.to_csv(f"./{filename.replace('.hdf5', '.csv')}", index=False)
            print(f'saved under {filename.replace(".hdf5", ".csv")}')

loading ./A_nodes.hdf5
saved under A_nodes.csv
loading ./C_nodes.hdf5
saved under C_nodes.csv
loading ./B_nodes.hdf5
saved under B_nodes.csv


In [2]:
df_A = pd.read_csv('A_nodes.csv', index_col=0)
df_B = pd.read_csv('B_nodes.csv', index_col=0)
df_C = pd.read_csv('C_nodes.csv', index_col=0)